In [2]:
import fitz  # PyMuPDF
import re
import csv
from pathlib import Path
import os

def is_footnote_by_lines(block, lines, x_range):
    """
    判斷文本塊是否位於特定線條下方並符合腳註格式
    """
    x0, y0, x1, y1, text = block[:5]
    text = text.strip()

    # 遍歷線條，檢查文本塊是否在線條下方
    for line in lines:
        line_start = line["items"][0][1]
        line_end = line["items"][0][2]

        # 檢查線條是否符合 x_range 限制
        if x_range[0] <= line_start.x <= x_range[1] and x_range[0] <= line_end.x <= x_range[1]:
            line_y = line_start.y
            if y0 > line_y:  # 文本塊位於線條下方
                # 符合腳註的正則格式
                if re.match(r'^[\d\*\+]', text):  
                    return True
    return False

def extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range):
    """
    提取以 'Opinion' 大標題開頭，'End of Document' 結尾的段落內容
    """
    with fitz.open(pdf_file_path) as doc:
        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["Page", "Paragraph"])  # CSV 標題

            is_within_opinion = False
            opinion_text = ""
            page_count = len(doc)

            for page_num in range(page_count):
                page = doc[page_num]
                text_blocks = page.get_text("blocks")  # 獲取文本塊
                lines = []

                # 搜尋頁面中的所有線條
                for item in page.get_drawings():
                    if item['type'] == 's' and item['color'] == color and width_range[0] <= item['width'] <= width_range[1]:
                        lines.append(item)
                
                lines.sort(key=lambda l: l['items'][0][1].y)  # 按線條的 Y 座標排序

                for block in text_blocks:
                    x0, y0, x1, y1, text = block[:5]
                    text = text.strip()

                    # 檢測 Opinion 開始標題
                    if "Opinion" in text and not is_within_opinion:
                        is_within_opinion = True
                        continue
                    
                    # 檢測 End of Document 結束標題
                    if "End of Document" in text and is_within_opinion:
                        is_within_opinion = False
                        break

                    # 如果在 Opinion 區域內，處理段落
                    if is_within_opinion:
                        # 排除腳註
                        if not is_footnote_by_lines(block, lines, x_range):
                            opinion_text += f"{text}\n"

                # 每處理一頁，將段落分割並寫入 CSV
                if opinion_text:
                    paragraphs = opinion_text.split("\n\n")
                    for paragraph in paragraphs:
                        if paragraph.strip():  # 確保段落非空
                            csv_writer.writerow([page_num + 1, paragraph.strip()])
                    opinion_text = ""  # 重置文本收集器

def process_folder(folder_path, output_folder_path, x_range, color, width_range):
    """
    遍歷資料夾中的 PDF 文件，提取 Opinion 區域並輸出到 CSV
    """
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_file_path = os.path.join(root, file)
                csv_file_name = file.replace('.pdf', '_opinion.csv').replace('.PDF', '_opinion.csv')
                csv_file_path = os.path.join(output_folder_path, csv_file_name)
                
                extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range)
                print(f"Processed {pdf_file_path}")

# 使用範例
data_folder_path = r'/Users/tangjiahong/Dropbox/textmining1/Data 1'
output_csv_folder_path = r'/Users/tangjiahong/Dropbox/textmining1/Opinion'

x_range = (50, 563)  # 定義線條的 X 座標範圍
color = (0.0, 0.0, 0.0)  # 定義線條顏色（黑色）on
width_range = (0.72, 0.73)  # 定義線條的寬
process_folder(data_folder_path, output_csv_folder_path, x_range, color, width_range)


Processed /Users/tangjiahong/Dropbox/textmining1/Data 1/12_17_1984-10_01_1991/1217 (6).pdf
MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

Processed /Users/tangjiahong/Dropbox/textmining1/Data 1/12_17_1984-10_01_1991/1217 (7).pdf
MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT

In [3]:
import fitz  # PyMuPDF
import re
import csv
from pathlib import Path

def is_footnote_by_lines(block, lines, x_range):
    """
    判斷文本塊是否位於特定線條下方並符合腳註格式
    """
    x0, y0, x1, y1, text = block[:5]
    text = text.strip()

    # 遍歷線條，檢查文本塊是否在線條下方
    for line in lines:
        line_start = line["items"][0][1]
        line_end = line["items"][0][2]

        # 檢查線條是否符合 x_range 限制
        if x_range[0] <= line_start.x <= x_range[1] and x_range[0] <= line_end.x <= x_range[1]:
            line_y = line_start.y
            if y0 > line_y:  # 文本塊位於線條下方
                # 符合腳註的正則格式
                if re.match(r'^[\d\*\+]', text):  
                    return True
    return False

def extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range):
    """
    提取以 'Opinion' 大標題開頭，'End of Document' 結尾的段落內容
    """
    with fitz.open(pdf_file_path) as doc:
        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["Page", "Paragraph"])  # CSV 標題

            is_within_opinion = False
            opinion_text = ""
            page_count = len(doc)

            for page_num in range(page_count):
                page = doc[page_num]
                text_blocks = page.get_text("blocks")  # 獲取文本塊
                lines = []

                # 搜尋頁面中的所有線條
                for item in page.get_drawings():
                    if item['type'] == 's' and item['color'] == color and width_range[0] <= item['width'] <= width_range[1]:
                        lines.append(item)
                
                lines.sort(key=lambda l: l['items'][0][1].y)  # 按線條的 Y 座標排序

                for block in text_blocks:
                    x0, y0, x1, y1, text = block[:5]
                    text = text.strip()

                    # 檢測 Opinion 開始標題
                    if "Opinion" in text and not is_within_opinion:
                        is_within_opinion = True
                        continue
                    
                    # 檢測 End of Document 結束標題
                    if "End of Document" in text and is_within_opinion:
                        is_within_opinion = False
                        break

                    # 如果在 Opinion 區域內，處理段落
                    if is_within_opinion:
                        # 排除腳註
                        if not is_footnote_by_lines(block, lines, x_range):
                            opinion_text += f"{text}\n"

                # 每處理一頁，將段落分割並寫入 CSV
                if opinion_text:
                    paragraphs = opinion_text.split("\n\n")
                    for paragraph in paragraphs:
                        if paragraph.strip():  # 確保段落非空
                            csv_writer.writerow([page_num + 1, paragraph.strip()])
                    opinion_text = ""  # 重置文本收集器

def process_folder(folder_path, output_folder_path, x_range, color, width_range):
    """
    遍歷資料夾中的 PDF 文件，提取 Opinion 區域並出到 CSV
    """
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_file_path = os.path.join(root, file)
                csv_file_name = file.replace('.pdf', '_opinion.csv').replace('.PDF', '_opinion.csv')
                csv_file_path = os.path.join(output_folder_path, csv_file_name)
                
                extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range)
                print(f"Processed {pdf_file_path}")

# 使用範例
data_folder_path = r'/Users/tangjiahong/Dropbox/textmining1/Data 2'
output_csv_folder_path = r'/Users/tangjiahong/Dropbox/textmining1/Opinion'

x_range = (50, 563)  # 定義線條的 X 座標範圍
color = (0.0, 0.0, 0.0)  # 定義線條顏色（黑色）on
width_range = (0.72, 0.73)  # 定義線條的寬度範圍

process_folder(data_folder_path, output_csv_folder_path, x_range, color, width_range)



MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file 

In [4]:
import fitz  # PyMuPDF
import re
import csv
from pathlib import Path

def is_footnote_by_lines(block, lines, x_range):
    """
    判斷文本塊是否位於特定線條下方並符合腳註格式
    """
    x0, y0, x1, y1, text = block[:5]
    text = text.strip()

    # 遍歷線條，檢查文本塊是否在線條下方
    for line in lines:
        line_start = line["items"][0][1]
        line_end = line["items"][0][2]

        # 檢查線條是否符合 x_range 限制
        if x_range[0] <= line_start.x <= x_range[1] and x_range[0] <= line_end.x <= x_range[1]:
            line_y = line_start.y
            if y0 > line_y:  # 文本塊位於線條下方
                # 符合腳註的正則格式
                if re.match(r'^[\d\*\+]', text):  
                    return True
    return False

def extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range):
    """
    提取以 'Opinion' 大標題開頭，'End of Document' 結尾的段落內容
    """
    with fitz.open(pdf_file_path) as doc:
        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["Page", "Paragraph"])  # CSV 標題

            is_within_opinion = False
            opinion_text = ""
            page_count = len(doc)

            for page_num in range(page_count):
                page = doc[page_num]
                text_blocks = page.get_text("blocks")  # 獲取文本塊
                lines = []

                # 搜尋頁面中的所有線條
                for item in page.get_drawings():
                    if item['type'] == 's' and item['color'] == color and width_range[0] <= item['width'] <= width_range[1]:
                        lines.append(item)
                
                lines.sort(key=lambda l: l['items'][0][1].y)  # 按線條的 Y 座標排序

                for block in text_blocks:
                    x0, y0, x1, y1, text = block[:5]
                    text = text.strip()

                    # 檢測 Opinion 開始標題
                    if "Opinion" in text and not is_within_opinion:
                        is_within_opinion = True
                        continue
                    
                    # 檢測 End of Document 結束標題
                    if "End of Document" in text and is_within_opinion:
                        is_within_opinion = False
                        break

                    # 如果在 Opinion 區域內，處理段落
                    if is_within_opinion:
                        # 排除腳註
                        if not is_footnote_by_lines(block, lines, x_range):
                            opinion_text += f"{text}\n"

                # 每處理一頁，將段落分割並寫入 CSV
                if opinion_text:
                    paragraphs = opinion_text.split("\n\n")
                    for paragraph in paragraphs:
                        if paragraph.strip():  # 確保段落非空
                            csv_writer.writerow([page_num + 1, paragraph.strip()])
                    opinion_text = ""  # 重置文本收集器

def process_folder(folder_path, output_folder_path, x_range, color, width_range):
    """
    遍歷資料夾中的 PDF 文件，提取 Opinion 區域並出到 CSV
    """
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_file_path = os.path.join(root, file)
                csv_file_name = file.replace('.pdf', '_opinion.csv').replace('.PDF', '_opinion.csv')
                csv_file_path = os.path.join(output_folder_path, csv_file_name)
                
                extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range)
                print(f"Processed {pdf_file_path}")

# 使用範例
data_folder_path = r'/Users/tangjiahong/Dropbox/textmining1/Data 3'
output_csv_folder_path = r'/Users/tangjiahong/Dropbox/textmining1/Opinion'

x_range = (50, 563)  # 定義線條的 X 座標範圍
color = (0.0, 0.0, 0.0)  # 定義線條顏色（黑色）on
width_range = (0.72, 0.73)  # 定義線條的寬度範圍

process_folder(data_folder_path, output_csv_folder_path, x_range, color, width_range)



Processed /Users/tangjiahong/Dropbox/textmining1/Data 3/07_07_2017-08_30_2021/0707 (10).pdf
MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

Processed /Users/tangjiahong/Dropbox/textmining1/Data 3/07_07_2017-08_30_2021/0707 (1).pdf
MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: F

# 加入Index

In [ ]:
import fitz  # PyMuPDF
import re
import csv
from pathlib import Path
import os

def is_footnote_by_lines(block, lines, x_range):
    """
    判斷文本塊是否位於特定線條下方並符合腳註格式
    """
    x0, y0, x1, y1, text = block[:5]
    text = text.strip()

    # 遍歷線條，檢查文本塊是否在線條下方
    for line in lines:
        line_start = line["items"][0][1]
        line_end = line["items"][0][2]

        # 檢查線條是否符合 x_range 限制
        if x_range[0] <= line_start.x <= x_range[1] and x_range[0] <= line_end.x <= x_range[1]:
            line_y = line_start.y
            if y0 > line_y:  # 文本塊位於線條下方
                # 符合腳註的正則格式
                if re.match(r'^[\d\*\+]', text):  
                    return True
    return False

def extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range):
    """
    提取以 'Opinion' 大標題開頭，'End of Document' 結尾的段落內容，並新增 Case Index 欄位
    """
    with fitz.open(pdf_file_path) as doc:
        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["Case Index", "Page", "Paragraph"])  # CSV 標題

            is_within_opinion = False
            opinion_text = ""
            case_index = 0  # 初始化判例索
            page_count = len(doc)

            for page_num in range(page_count):
                page = doc[page_num]
                text_blocks = page.get_text("blocks")  # 獲取文本塊
                lines = []

                # 搜尋頁面中的所有線條
                for item in page.get_drawings():
                    if item['type'] == 's' and item['color'] == color and width_range[0] <= item['width'] <= width_range[1]:
                        lines.append(item)
                
                lines.sort(key=lambda l: l['items'][0][1].y)  # 按線條的 Y 座標排序

                for block in text_blocks:
                    x0, y0, x1, y1, text = block[:5]
                    text = text.strip()

                    # 檢測 Opinion 開始標題
                    if "Opinion" in text and not is_within_opinion:
                        is_within_opinion = True
                        case_index += 1  # 新增一個判例索引
                        continue
                    
                    # 檢測 End of Document 結束標題
                    if "End of Document" in text and is_within_opinion:
                        is_within_opinion = False
                        break

                    # 如果在 Opinion 區域內，處理段落
                    if is_within_opinion:
                        # 排除腳註
                        if not is_footnote_by_lines(block, lines, x_range):
                            opinion_text += f"{text}\n"

                # 每處理一頁，將段落分割並寫入 CSV
                if opinion_text:
                    paragraphs = opinion_text.split("\n\n")
                    for paragraph in paragraphs:
                        if paragraph.strip():  # 確保段落非空
                            csv_writer.writerow([case_index, page_num + 1, paragraph.strip()])
                    opinion_text = ""  # 重置文本收集器

def process_folder(folder_path, output_folder_path, x_range, color, width_range):
    """
    遍歷資料夾中的 PDF 文件，提取 Opinion 區域並輸出到 CSV
    """
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_file_path = os.path.join(root, file)
                csv_file_name = file.replace('.pdf', '_opinion.csv').replace('.PDF', '_opinion.csv')
                csv_file_path = os.path.join(output_folder_path, csv_file_name)
                
                extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range)
                print(f"Processed {pdf_file_path}")

# 使用範例ㄟ
data_folder_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1'
output_csv_folder_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Opinion'

x_range = (50, 563)  # 定義線條的 X 座標範圍
color = (0.0, 0.0, 0.0)  # 定義線條顏色（黑色）
width_range = (0.72, 0.73)  # 定義線條的寬度範圍

process_folder(data_folder_path, output_csv_folder_path, x_range, color, width_range)


MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file format

MuPDF error: library error: zlib error: unknown compression method

MuPDF error: library error: FT_New_Memory_Face(Arial): unknown file 

In [1]:
import fitz  # PyMuPDF
import re
import csv
from pathlib import Path
import os

def is_footnote_by_lines(block, lines, x_range):
    """
    判斷文本塊是否位於特定線條下方並符合腳註格式
    """
    x0, y0, x1, y1, text = block[:5]
    text = text.strip()

    # 遍歷線條，檢查文本塊是否在線條下方
    for line in lines:
        if "items" in line and line["items"]:
            line_start = line["items"][0][1]
            line_end = line["items"][0][2]

            # 檢查線條是否符合 x_range 限制
            if x_range[0] <= line_start.x <= x_range[1] and x_range[0] <= line_end.x <= x_range[1]:
                line_y = line_start.y
                if y0 > line_y:  # 文本塊位於線條下方
                    # 符合腳註的正則格式
                    if re.match(r'^[\d\*\+\-\(\)]+', text):  
                        return True
    return False

def extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range):
    """
    提取以 'Opinion' 大標題開頭，'End of Document' 結尾的段落內容，並新增 Case Index 欄位
    """
    with fitz.open(pdf_file_path) as doc:
        with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(["Case Index", "Page", "Paragraph"])  # CSV 標題

            is_within_opinion = False
            opinion_text = ""
            case_index = 0  # 初始化判例索引
            page_count = len(doc)

            for page_num in range(page_count):
                page = doc[page_num]
                text_blocks = page.get_text("blocks")  # 獲取文本塊
                lines = []

                # 搜尋頁面中的所有線條
                for item in page.get_drawings():
                    if item['type'] == 's' and item['color'] == color and width_range[0] <= item['width'] <= width_range[1]:
                        lines.append(item)
                
                lines.sort(key=lambda l: l['items'][0][1].y if "items" in l and l["items"] else float('inf'))  # 按線條的 Y 座標排序

                for block in text_blocks:
                    x0, y0, x1, y1, text = block[:5]
                    text = text.strip()

                    # 檢測 Opinion 開始標題
                    if re.search(r'\bOpinion\b', text, re.IGNORECASE) and not is_within_opinion:
                        is_within_opinion = True
                        case_index += 1  # 新增一個判例索引
                        continue
                    
                    # 檢測 End of Document 結束標題
                    if re.search(r'End of Document', text, re.IGNORECASE) and is_within_opinion:
                        is_within_opinion = False
                        break

                    # 如果在 Opinion 區域內，處理段落
                    if is_within_opinion:
                        # 排除腳註
                        if not is_footnote_by_lines(block, lines, x_range):
                            opinion_text += f"{text}\n"

                # 每處理一頁，將段落分割並寫入 CSV
                if opinion_text:
                    paragraphs = opinion_text.split("\n\n")
                    for paragraph in paragraphs:
                        if paragraph.strip():  # 確保段落非空
                            csv_writer.writerow([case_index, page_num + 1, paragraph.strip()])
                    opinion_text = ""  # 重置文本收集器

def process_folder(folder_path, output_folder_path, x_range, color, width_range):
    """
    遍歷資料夾中的 PDF 文件，提取 Opinion 區域並輸出到 CSV
    """
    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_file_path = os.path.join(root, file)
                csv_file_name = file.replace('.pdf', '_opinion.csv').replace('.PDF', '_opinion.csv')
                csv_file_path = os.path.join(output_folder_path, csv_file_name)
                
                extract_opinion_content(pdf_file_path, csv_file_path, x_range, color, width_range)
                print(f"Processed {pdf_file_path}")

# 主程序
data_folder_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court'
output_csv_folder_path = r'C:\Users\User\Dropbox\textmining1\Opinion_Supreme'

x_range = (50, 563)  # 定義線條的 X 座標範圍
color = (0.0, 0.0, 0.0)  # 定義線條顏色（黑色）
width_range = (0.72, 0.73)  # 定義線條的寬度範圍

process_folder(data_folder_path, output_csv_folder_path, x_range, color, width_range)


Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (1).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (10).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (11).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (12).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (13).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (14).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (15).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (16).PDF
Processed C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\Context Samples_US Supreme Court\Supreme (1

In [3]:
import re
import fitz  # PyMuPDF
import csv

opinion_pattern = r'Opinion'
end_pattern = r'End of Document'
pdf_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\01_01_1891-01_30_1978\0101 (1).pdf'
csv_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Data 1\01_01_1891-01_30_1978\0101.csv'
doc = fitz.open(pdf_path)
# Open the PDF and the CSV file
with doc, open(csv_path, mode='w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['pdf', 'page', 'content'])  # Write header row
    
    opinion_started = False  # Flag to track if opinion collection should start
    
    # Loop through each page in the specified range
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_text = ""  # Text container for opinion content on the current page
        
        # Use 'dict' mode to access font and size metadata
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if "lines" in block:
                for line in block["lines"]:
                    for span in line["spans"]:
                        text = span["text"].strip()
                        
                        # Step 3: Check for the start of the "Opinion" section
                        if span['size'] == 14.0 and span['font'] == 'Helvetica-Bold' and re.search(opinion_pattern, text):
                            opinion_started = True
                            page_text = ""  # Reset the page text for new opinion section
                            continue
                        
                        # Step 4 and 5: Collect text after "Opinion" begins, meeting the size/font requirements
                        if opinion_started and span['size'] == 10 and (
                            span['font'] == 'Helvetica' or span['font'] == 'Helvetica-BoldOblique'
                        ):
                            page_text += " " + text
                        
                        # Step 6: Stop collecting text if "End of Document" is found
                        if opinion_started and re.search(end_pattern, text):
                            page_text = re.sub(end_pattern, '', page_text)  # Remove "End of Document"
                            opinion_started = False  # Reset flag to stop collecting
                            
                            # Write collected text for this page into the CSV
                            csv_writer.writerow([pdf_path, page_num, page_text.strip()])
                            page_text = ""  # Clear page_text for the next page
                            break  # Stop further processing on this page if the end is found
        
        # Write page content to CSV if opinion section continues across multiple pages
        if opinion_started and page_text:
            csv_writer.writerow([pdf_path, page_num+1, page_text.strip()])